In [1]:
import os
import json
import pandas as pd
import requests
from datetime import datetime
from dotenv import load_dotenv
from utils import check_env_variable, check_file_exists, generate_tp_name
from inspect import cleandoc

from gptravel.core.travel_planner.openai_engine import ChatGPTravelEngine
from gptravel.core.travel_planner.prompt import PlainTravelPrompt
from gptravel.core.travel_planner.travel_engine import TravelPlanJSON
from gptravel.core.services.geocoder import GeoCoder

from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_core.pydantic_v1 import BaseModel
from serpapi import GoogleSearch
from agents import get_flight_query_agent, get_travel_plan_generator_agent, get_accomodation_selection_chain, get_flight_selection_chain

from gptravel.core.utils.regex_tool import JsonExtractor
from utils import from_json_to_text
from typing import Dict, Any, Optional, List, Tuple
from operator import itemgetter

## pydantic schemas
from travel_plan_schema import TravelPlan

load_dotenv()

check_env_variable('OPENAI_API_KEY')
examples_folder = os.path.join('examples')

OPENAI_API_KEY loaded correctly


In [2]:
PARAMETERS = {
    'departure_place': 'Milan',
    'departure_date_dt': datetime(2024, 11, 30),
    'destination_place': 'Berlin',
    'return_date_dt': datetime(2024, 12, 3),
    'gpt_params': {
        'model': 'gpt-4o-mini',
        #'max_tokens': 600,
        'temperature': 0,
    },
    'user_travel_notes': 'I would like to organize a romantic travel to surprise my girlfirend. I would like to leave Berlin in the morning the last day',
}

FLIGHT_API_PARAMETERS = {
    'engine': 'google_flights',
    'gl': 'it', # country of the engine search
    'hl': 'en', # language
    'currency': 'EUR',
    'type': '2', # 1 = round trip, 2 = one way, 3 = multicity
    'travel_class': 1, # 1 = economy, 2 = premium economy, 3 = business, 4 = first
    'adults': 2, # number of adults
    'children': 0, # number of children
    'stops': 0, # number of scali
    'bags': 0, # number of carry bags
    'api_key': os.getenv('SERP_API_KEY'),
}

HOTELS_API_PARAMETERS = {
    "currency": "EUR",
    "sort_by": "3",
    'api_key': os.getenv('SERP_API_KEY'),
    "engine": "google_hotels"
}


FLIGHTS_ENDPOINTS = ['https://serpapi.com/searches/5b21533b2c3e8273/66d8906ffc183d0799f96b3b.json',
             'https://serpapi.com/searches/5b21533b2c3e8273/66d89070708e2e2145e0af19.json']

HOTELS_ENDPOINT = ['https://serpapi.com/searches/f29526eb14e9aac7/66dc83b120e6eb7e4e0353a4.json']

PARAMETERS['n_travel_days'] = (PARAMETERS['return_date_dt'] - PARAMETERS['departure_date_dt']).days + 1
PARAMETERS['departure_date'] = PARAMETERS['departure_date_dt'].strftime('%Y-%m-%d')
PARAMETERS['return_date'] = PARAMETERS['return_date_dt'].strftime('%Y-%m-%d')
print(PARAMETERS)

{'departure_place': 'Milan', 'departure_date_dt': datetime.datetime(2024, 11, 30, 0, 0), 'destination_place': 'Berlin', 'return_date_dt': datetime.datetime(2024, 12, 3, 0, 0), 'gpt_params': {'model': 'gpt-4o-mini', 'temperature': 0}, 'user_travel_notes': 'I would like to organize a romantic travel to surprise my girlfirend. I would like to leave Berlin in the morning the last day', 'n_travel_days': 4, 'departure_date': '2024-11-30', 'return_date': '2024-12-03'}


# Travel Plan Generation

In [3]:
%%time

travel_plan_generator_agent = get_travel_plan_generator_agent(input_parameters=PARAMETERS)

generated_travel_plan = travel_plan_generator_agent.invoke(PARAMETERS)

print(generated_travel_plan.json())

{"day_plans": [{"date": "2024-11-30", "activities": [{"time": "2024-11-30T14:00:00", "place": "Brandenburg Gate", "description": "Start your romantic trip with a visit to the iconic Brandenburg Gate. Take some beautiful photos together."}, {"time": "2024-11-30T16:00:00", "place": "Museum Island", "description": "Explore the museums on Museum Island. Choose one or two museums to visit and enjoy the art and history."}, {"time": "2024-11-30T19:00:00", "place": "Restaurant Tim Raue", "description": "Enjoy a romantic dinner at the Michelin-starred Restaurant Tim Raue. Make sure to book a table in advance."}], "city": "Berlin"}, {"date": "2024-12-01", "activities": [{"time": "2024-12-01T10:00:00", "place": "Berlin Cathedral", "description": "Visit the Berlin Cathedral and climb to the top for a stunning view of the city."}, {"time": "2024-12-01T12:00:00", "place": "Gendarmenmarkt", "description": "Stroll around Gendarmenmarkt, one of the most beautiful squares in Berlin. Enjoy a coffee at on

In [4]:
PARAMETERS['generated_travel_plan'] = generated_travel_plan # must be initialized with a RunnableAssign when building the whole chain
PARAMETERS['generated_travel_plan_json'] = generated_travel_plan.json()

# Flight search

In [5]:
flight_selector_agent = get_flight_selection_chain(input_parameters=PARAMETERS,
                                              serp_api_default_pars=FLIGHT_API_PARAMETERS,
                                              hardcoded_endpoints=FLIGHTS_ENDPOINTS)


In [6]:
selected_flights = flight_selector_agent.invoke(PARAMETERS)

Reading from direct endpoint
Reading from direct endpoint


In [7]:
selected_flights

FlightDetails(flights=[Flight(departure_airport=Airport(name='Milan Malpensa Airport', airport_id='MXP', time='2024-11-30 16:10'), arrival_airport=Airport(name='Berlin Brandenburg Airport (BER)', airport_id='BER', time='2024-11-30 17:55'), duration=105, airline='Ryanair', travel_class=<FlightClass.economy: 'economy'>, flight_number='FR1234', price=47, reason='This flight allows you to arrive in Berlin in the afternoon, giving you time to settle in before your romantic evening.'), Flight(departure_airport=Airport(name='Berlin Brandenburg Airport (BER)', airport_id='BER', time='2024-12-03 08:00'), arrival_airport=Airport(name='Milan Malpensa Airport', airport_id='MXP', time='2024-12-03 09:45'), duration=105, airline='Ryanair', travel_class=<FlightClass.economy: 'economy'>, flight_number='FR5678', price=42, reason='This morning flight allows you to leave Berlin early, fitting perfectly with your preference to leave in the morning.')])

# Hotel search

In [8]:
accomodations_selector_agent = get_accomodation_selection_chain(input_parameters=PARAMETERS,
                                                           serp_api_default_pars=HOTELS_API_PARAMETERS,
                                                           hardcoded_endpoints=HOTELS_ENDPOINT)
selected_accomodation = accomodations_selector_agent.invoke(PARAMETERS)

Reading from direct endpoint


In [9]:
selected_accomodation

AccomodationstDetails(accomodations=[HotelDetails(type='hotel', name='Metropol Hostel Berlin', description='Relaxed budget lodging offering rooms & dorms with free Wi-Fi & breakfast, plus a lobby bar.', link='https://neu.metropolhostel-berlin.de/', gps_coordinates=GpsCoordinates(latitude=52.4937981, longitude=13.387845599999999, city='Berlin'), check_in_time='3:00\u202fPM', check_out_time='10:00\u202fAM', rate_per_night=Rate(lowest='€72', extracted_lowest=72, before_taxes_fees='€31', extracted_before_taxes_fees=31), total_rate=Rate(lowest='€216', extracted_lowest=216, before_taxes_fees='€92', extracted_before_taxes_fees=92), nearby_places=[NearbyPlace(name='Brandenburg Gate', transportations=[Transportation(type_='Taxi', duration='9 min')]), NearbyPlace(name='Südkreuz', transportations=[Transportation(type_='Taxi', duration='8 min')]), NearbyPlace(name='Berlin Brandenburg Airport', transportations=[Transportation(type_='Taxi', duration='24 min'), Transportation(type_='Public transport'